# 案件
- 【市場調査】Amazon/ヤフオクの市場の調査をお願いします。初心者歓迎/隙間時間にもできます
- https://crowdworks.jp/public/jobs/9151993

# 動作検証

## ヤフオク情報取得

In [91]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome import service as fs
from time import sleep
from bs4 import BeautifulSoup
import datetime
import csv
import pandas as pd

CHROMEDRIVER = r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\chromedriver.exe'
chrome_service = fs.Service(executable_path = CHROMEDRIVER)
options = Options()
# options.add_argument('--headless')  # ヘッドレスモード
driver = webdriver.Chrome(service = chrome_service, options = options)

url = 'https://auctions.yahoo.co.jp/jp/show/rating?userID=k_h_112&role=seller'
driver.get(url)

In [92]:
# 一覧ページの HTML 構造を取得する
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [58]:
# 各商品の親要素を取得する
items = soup.select('td[colspan] > table[bgcolor="#ffffff"]')

# 取得結果の格納用リストを準備する
yafuoku_item_list = []

In [80]:
# for item in items:

detail_url = item.select('a')[0]['href']
print(detail_url)

name = item.select('a')[0].text
print(name)

eval_date = item.select('td[width="98%"] small')[0].text.splitlines()[1]
print(hyouka_date)

https://page.auctions.yahoo.co.jp/jp/auction/r1086528965
ホンダ ズーマー カスタム ヘッドライト ガード カバー
(評価日時：1か月以内)


In [71]:
# 詳細ページを開く
driver.get(detail_url)

In [72]:
# 詳細ページの HTML 構造を取得する
soup_detail = BeautifulSoup(driver.page_source, 'html.parser')

In [74]:
price = soup_detail.select('dd.Price__value')[0].text.strip()
print(price)

870円（税 0 円）


In [75]:
# 送料の詳細ポップアップを開く
driver.find_element(By.ID, 'postageDetailCurrent').click()

In [77]:
# ポップアップの中から送料を取得する　→　送料を取れるようになった！！！
postage = soup_detail.select('div.BidModal__postagePrice')[0].text
print(postage)

500円（税込）


In [78]:
# ポップアップを閉じる
driver.find_element(By.XPATH, '//a[@class="BidModal__hideButton BidModal__hideButton--large js-modal-hide cl-nofollow"]').click()

In [81]:
# 取得した情報をリストに追加する
yafuoku_item_list.append([name, price, postage, eval_date, detail_url])
sleep(1)

In [84]:
# 取得結果を保存する
csv_header = ['商品名', '価格', '送料', '評価日時', '商品ページURL']
csv_date = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
csv_file_name = 'ヤフオクデータ_' + csv_date + '.csv'

with open(csv_file_name, 'w', errors = 'ignore') as file:
    writer = csv.writer(file, lineterminator='\n')
    writer.writerow(csv_header)
    writer.writerows(yafuoku_item_list)

In [110]:
print(soup.select('a:contains("次の25件")')[0]['href'])

https://auctions.yahoo.co.jp/jp/show/rating?userID=mlamb63401&role=seller&apg=2


In [95]:
driver.find_element(By.XPATH, '(//a[contains(text(),"次の25件")])[1]').click()

## Amazon情報取得

In [35]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome import service as fs
from time import sleep
from bs4 import BeautifulSoup
import datetime
import csv
import pandas as pd

CHROMEDRIVER = r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\chromedriver.exe'
chrome_service = fs.Service(executable_path = CHROMEDRIVER)
options = Options()
# options.add_argument('--headless')  # ヘッドレスモード
driver = webdriver.Chrome(service = chrome_service, options = options)

# Amazon のトップページを開く
url = 'https://www.amazon.co.jp/ref=nav_logo'
driver.get(url)

In [36]:
yafuoku_name = 'Sweetwinds ペニスリング コックリング 二重リング 日本人サイズ 柔らかいシリコン 高弾力 男性用オナニーリング アダル'

driver.find_element(By.ID, 'twotabsearchtextbox').send_keys(yafuoku_name)
driver.find_element(By.ID, 'nav-search-submit-button').click()

In [37]:
# ■ アダルト商品の場合、年齢確認のポップアップ（別ウィンドウ）が表示されるので「はい」をクリックする
driver.find_element(By.ID, 'a-autoid-1-announce').click()

In [38]:
# 商品ページのリンクをクリックする。別ウィンドウで開かれる
driver.find_element(By.XPATH, f'(//span[contains(text(),"{yafuoku_name}")]/parent::a[@target="_blank"])[1]').click()

In [40]:
# 開いているすべてのウィンドウハンドルを取得する（リスト形式）
handle_array = driver.window_handles
# 別ウィンドウに切り替える
driver.switch_to.window(handle_array[-1])
# 別ウィンドウの HTML 構造を取得する
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [ ]:
amazon_name = soup.select('span#productTitle')[0].text.strip()
print('Amazon商品名:', amazon_name)

amazon_price = soup.select('div#corePrice_feature_div span.a-price-whole')[0].text
# //div[@id="apex_desktop"]//span[@id="sns-base-price"]/text()  # メインの価格
# //div[@id="corePrice_feature_div"]//span[@class="a-offscreen"]  # 通常の注文
print('Amazon価格:', amazon_price)

amazon_url = driver.current_url
print('Amazon商品URL:', amazon_url)

amazon_info_get_date = datetime.datetime.now()
print('Amazon情報取得日:', amazon_info_get_date)

In [103]:
print(soup_detail.select('span:contains("件")')[0].text)

3件


# 完成版
1. ヤフオクから情報を取得する
1. ヤフオクで取得した情報を基に Amazon で検索する

## ヤフオクの情報を取得する

In [111]:
%%time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome import service as fs
from time import sleep
from bs4 import BeautifulSoup
import sys
import datetime
import csv
import pandas as pd

CHROMEDRIVER = r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\chromedriver.exe'
chrome_service = fs.Service(executable_path = CHROMEDRIVER)
options = Options()
# options.add_argument('--headless')  # ヘッドレスモード
driver = webdriver.Chrome(service = chrome_service, options = options)

# 調査対象セラーの URL
# url = 'https://auctions.yahoo.co.jp/jp/show/rating?userID=k_h_112&role=seller'
url = 'https://auctions.yahoo.co.jp/jp/show/rating?userID=mlamb63401&role=seller'

# 取得結果の格納用リストを準備する
yafuoku_item_list = []

for i in range(8):
    print(f'▼ {i+1}ページ目の情報を取得します ▼')
    print('')

    driver.get(url)
    sleep(1)

    # 一覧ページの HTML 構造を取得する
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # 各商品の親要素を取得する
    items = soup.select('td[colspan] > table[bgcolor="#ffffff"]')

    # 一覧ページ内の商品情報をすべて取得する
    for item in items:
        try:
            # 商品名　一覧ページから取得
            name = item.select('a')[0].text
            print(name)

            # 評価日　一覧ページから取得
            eval_date = item.select('td[width="98%"] small')[0].text.splitlines()[1]
            print(eval_date)

            # 詳細ページへの URL
            detail_url = item.select('a')[0]['href']
            print(detail_url)

            # 詳細ページを開く
            # ■ アダルトカテゴリを開こうとすると年齢認証（ログイン認証）が走り取得できない
            driver.get(detail_url)

            # 詳細ページの HTML 構造を取得する
            soup_detail = BeautifulSoup(driver.page_source, 'html.parser')

            # 価格
            price = soup_detail.select('dd.Price__value')[0].text.strip()
            print(price)

            # 入札件数
            bid_num = soup_detail.select('span:contains("件")')[0].text
            print(bid_num)

            # 送料の詳細ポップアップを開く
            driver.find_element(By.ID, 'postageDetailCurrent').click()
            sleep(1)

            # ポップアップの中から送料を取得する　→　送料を取れるようになった！！！
            postage = soup_detail.select('div.BidModal__postagePrice')[0].text
            print(postage)

            # ポップアップを閉じる
            driver.find_element(By.XPATH, '//a[@class="BidModal__hideButton BidModal__hideButton--large js-modal-hide cl-nofollow"]').click()
            sleep(1)

            # 取得した情報をリストに追加する
            yafuoku_item_list.append([name, price, postage, bid_num, eval_date, detail_url])
            print('-----')
            # sleep(1)
        except:
            print('◆ 取得に失敗しました ◆')
            # 取得した情報をリストに追加する
            yafuoku_item_list.append([name, '取得不可', '取得不可', '取得不可', eval_date, detail_url])
            print('-----')
            
    try:
        url = soup.select('a:contains("次の25件")')[0]['href']
        sleep(1)
    except:
        driver.quit()
        break

# 取得結果を保存する
csv_header = ['商品名', '価格', '送料', '入札件数', '評価日時', '商品ページURL']
csv_date = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
csv_file_name = 'ヤフオクデータ_' + csv_date + '.csv'

with open(csv_file_name, 'w', errors = 'ignore') as file:
    writer = csv.writer(file, lineterminator='\n')
    writer.writerow(csv_header)
    writer.writerows(yafuoku_item_list)

● 1件目から25件目の情報を取得します ●
FKS マウントヘッド ステンレス タイヤチェンジャー タイヤ タイヤレバー オイルチェンジャー パンタジャッキ リムガード (シ
(評価日時：1週間以内)
https://page.auctions.yahoo.co.jp/jp/auction/x1087885486
4,882円（税込 5,370 円）
1件
0円（税込）
-----
Shileded ハロウィーン 飾り 風船セット パーティー 飾り アルミバルーン ガーランド バナー ハニカム飾り コウモリ か
(評価日時：1週間以内)
https://page.auctions.yahoo.co.jp/jp/auction/b1081825363
3,055円（税込 3,360 円）
1件
0円（税込）
-----
Bowarepro 40PCS エアコンバルブコア 虫回しドライバー エアバルブレンチ 車A/C R12 R134a冷媒交換 修理
(評価日時：1週間以内)
https://page.auctions.yahoo.co.jp/jp/auction/n1087870548
3,336円（税込 3,669 円）
1件
0円（税込）
-----
丸型バックミラー 左右セット 8mm 正ネジ （3種類の取り付けボルト付属）スーパーカブ スクーピー TACT（タクト）Dio（デ
(評価日時：1週間以内)
https://page.auctions.yahoo.co.jp/jp/auction/r1087874306
3,024円（税込 3,326 円）
1件
0円（税込）
-----
キックスターターレバー キックペダル キックペダルレバー ゴム+鋼合金材質 オートバイ改造部品 長さ280mm 取付穴径16mm
(評価日時：1週間以内)
https://page.auctions.yahoo.co.jp/jp/auction/m1087880566
4,582円（税込 5,040 円）
1件
0円（税込）
-----
h&b 汎用 燃料 タンク キャップ ホンダ ジェイド250 CBX125カスタム ナイトホーク250 用 鍵2本付き 社外
(評価日時：1週間以内)
https://page.auctions.yahoo.co.jp/jp/auction/10

## ヤフオクの取得データを基に Amazon の情報を検索する

In [42]:
%%time  # 283件：Wall time: 27min 54s
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome import service as fs
from time import sleep
from bs4 import BeautifulSoup
import datetime
import csv
import pandas as pd

CHROMEDRIVER = r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\chromedriver.exe'
chrome_service = fs.Service(executable_path = CHROMEDRIVER)
options = Options()
# options.add_argument('--headless')  # ヘッドレスモード
driver = webdriver.Chrome(service = chrome_service, options = options)

# Amazon のトップページを開く
url = 'https://www.amazon.co.jp/ref=nav_logo'
driver.get(url)
sleep(1)

# 大元のウィンドウの保持
original_window = driver.current_window_handle

# ヤフオクデータの取得
df = pd.read_csv('yafuoku_seller_crawl.csv', encoding='cp932')

# 取得情報格納用のリスト準備
amazon_list = []

for index, yafuoku_item in df.iterrows():
    try:
        yafuoku_name = yafuoku_item['name']
        print('ヤフオク商品名:', yafuoku_name)

        yafuoku_price = yafuoku_item['price']
        # print('ヤフオク価格:', yafuoku_price)
        
        yafuoku_tax = yafuoku_item['tax']
        # print('ヤフオク税金:', yafuoku_tax)

        yafuoku_postage = yafuoku_item['postage']
        # print('ヤフオク送料:', yafuoku_postage)
        
        yafuoku_postage_due = yafuoku_item['postage_due']
        # print('ヤフオク送料負担:', yafuoku_postage_due)
        
        yafuoku_free_postage = yafuoku_item['free_postage']
        # print('ヤフオク送料無料:', yafuoku_free_postage)
        
        yafuoku_url = yafuoku_item['url']
        # print('ヤフオク商品URL:', yafuoku_url)
        
        yafuoku_end_date = yafuoku_item['end_date']
        # print('ヤフオク終了日:', yafuoku_end_date)
        
        yafuoku_info_get_date = yafuoku_item['info_get_date']
        # print('ヤフオク情報取得日:', yafuoku_info_get_date)

        # 検索ボックスに商品名を入力する
        driver.find_element(By.ID, 'twotabsearchtextbox').send_keys(yafuoku_name)
        # 検索ボタンをクリックする
        driver.find_element(By.ID, 'nav-search-submit-button').click()
        sleep(1)

        # アダルト商品の場合、年齢確認のポップアップ（別ウィンドウ）が表示されるので「はい」をクリックする
        # if driver.find_element(By.ID, 'a-autoid-1-announce'):
        #     driver.find_element(By.ID, 'a-autoid-1-announce').click()
        #     sleep(1)
        
        # 商品ページのリンクをクリックする。別ウィンドウで開かれる
        driver.find_element(By.XPATH, f'(//span[contains(text(),"{yafuoku_name}")]/parent::a[@target="_blank"])[1]').click()

        # 開いているすべてのウィンドウハンドルを取得する（リスト形式）
        handle_array = driver.window_handles
        # 別ウィンドウに切り替える
        driver.switch_to.window(handle_array[-1])
        # 別ウィンドウの HTML 構造を取得する
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        amazon_name = soup.select('span#productTitle')[0].text.strip()
        print('Amazon商品名:', amazon_name)

        amazon_price = soup.select('div#corePrice_feature_div span.a-price-whole')[0].text
        # print('Amazon価格:', amazon_price)

        amazon_url = driver.current_url
        # print('Amazon商品URL:', amazon_url)
        
        amazon_info_get_date = datetime.datetime.now()
        # print('Amazon情報取得日:', amazon_info_get_date)

        # 検索対象商品名と取得商品名が一致しているかをチェック
        if yafuoku_name == amazon_name:
            check = '一致'
        else:
            check = '【商品名不一致】'
        # print('商品名整合性:', check)

        # 取得した情報をリストに追加する
        amazon_list.append([
            yafuoku_name, yafuoku_price, yafuoku_tax, yafuoku_postage, yafuoku_postage_due, yafuoku_free_postage, yafuoku_url, yafuoku_end_date, yafuoku_info_get_date, \
            amazon_name, amazon_price, amazon_url, amazon_info_get_date, check
        ])

        # 現在のウィンドウを閉じる
        driver.close()
        # ウィンドウハンドラを大元のウィンドウに切り替える
        driver.switch_to.window(original_window)

        # 検索ボックスに残っている文字列を消す
        driver.find_element(By.ID, 'twotabsearchtextbox').clear()
    except:
        print('【Amazon情報を取得できませんでした】')
        check = '【Amazon情報未取得】'
        amazon_list.append([
            yafuoku_name, yafuoku_price, yafuoku_tax, yafuoku_postage, yafuoku_postage_due, yafuoku_free_postage, yafuoku_url, yafuoku_end_date, yafuoku_info_get_date, \
            '【取得不可】', '-', '-', '-', check
        ])
        driver.find_element(By.ID, 'twotabsearchtextbox').clear()
    finally:
        print('')

# 取得した全データを csv ファイルに出力する
csv_header = [
    'ヤフオク商品名', 'ヤフオク価格', 'ヤフオク税金', 'ヤフオク送料', 'ヤフオク送料負担', 'ヤフオク送料無料', 'ヤフオク商品URL', 'ヤフオク終了日', 'ヤフオク情報取得日', \
    'Amazon商品名', 'Amazon価格', 'Amazon商品URL', 'Amazon情報取得日', '商品名整合性'
]
csv_file_name = 'yafuoku_amazon_research.csv'

with open(csv_file_name, 'w', errors = 'ignore') as file:
    writer = csv.writer(file, lineterminator='\n')
    writer.writerow(csv_header)
    writer.writerows(amazon_list)
    
print('----- 取得完了 -----')

ヤフオク商品名: 座るだけで骨盤 背骨 矯正効果 クッション ブルー
【Amazon情報を取得できませんでした】

ヤフオク商品名: ABSプラスチック製　カー　フェイク　ダンプ　電子　ターボ　ブロー　オフ　フーター　バルブ　 アナログ　サウンド
Amazon商品名: 【ノーブランド 品】ABSプラスチック製　カー　フェイク　ダンプ　電子　ターボ　ブロー　オフ　フーター　バルブ　 アナログ　サウンド

ヤフオク商品名: 卓上マイクスタンド+ポップブロッカー 伸縮式&デスクアーム式
【Amazon情報を取得できませんでした】

ヤフオク商品名: h&b 汎用 燃料 タンク キャップ ホンダ ジェイド250 CBX125カスタム ナイトホーク250 用 鍵2本付き 社外
Amazon商品名: h&b 汎用 燃料 タンク キャップ ホンダ ジェイド250 CBX125カスタム ナイトホーク250 用 鍵2本付き 社外 楕円型

ヤフオク商品名: 時計バンド 交換ベルトシリコーンゴム 腕時計ストラップ 20mm ブラック
Amazon商品名: 時計バンド 交換ベルトシリコーンゴム 腕時計ストラップ 20mm ブラック

ヤフオク商品名: nan
【Amazon情報を取得できませんでした】

ヤフオク商品名: ユニバーサル 基板 両面 PCB回路基板 PCB ユニバーサル プリント基板 DIY はんだ付け 2 * 8CM ・3 * 7CM ・4 * 6CM ・5 * 7CM（32個）
【Amazon情報を取得できませんでした】

ヤフオク商品名: トヨタ ヤリスクロス GRヤリス対応 オートブレーキホールドキット エンラージ商事
【Amazon情報を取得できませんでした】

ヤフオク商品名: TOYOTA (トヨタ) 純正部品 エアピュリファイヤ フィルタ 品番88926-22340
Amazon商品名: TOYOTA (トヨタ) 純正部品 エアピュリファイヤ フィルタ 品番88926-22340

ヤフオク商品名: Honda(ホンダ) 純正サブエアークリーナーフィルター CB1000SF X4 CB1300SFホーネット250/600 ジェイド CB400SF/SB CBR1000F CBR250RR/400RR
Amazon商品名: Honda(ホンダ) 純正サブエアー